## XPT TO CSV Notebook

### read XPTs

In [1]:
import pandas as pd
from pathlib import Path

data_dir = Path("../data")
raw_data_dir = data_dir / "raw"
clean_data_dir = data_dir / "clean"

clean_data_dir.mkdir(parents=True, exist_ok=True)

def read_xpt(file_name, limit_cols=None):
    xpt_file_path = raw_data_dir / file_name
    df = pd.read_sas(xpt_file_path, format='xport')
    if limit_cols:
        cols_to_keep = ['SEQN'] + [col for col in limit_cols if col != 'SEQN']
        existing_cols_to_keep = [col for col in cols_to_keep if col in df.columns]
        missing_cols = set(cols_to_keep) - set(existing_cols_to_keep)
        if missing_cols:
                print(f"Warning: For {file_name}, specified columns not found and will be ignored: {missing_cols}")
        return df[existing_cols_to_keep]
    return df

In [2]:
# sanity check
df_demo_raw = read_xpt("DEMO_J.XPT")
print(f"Successfully read {df_demo_raw}")
print("First 5 rows of the DEMO_J.XPT data:")
print(df_demo_raw.head())

# Display some information about the DataFrame
print("\nDataFrame Info:")
df_demo_raw.info()


Successfully read           SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDRETH1  \
0      93703.0      10.0       2.0       2.0       2.0       NaN       5.0   
1      93704.0      10.0       2.0       1.0       2.0       NaN       3.0   
2      93705.0      10.0       2.0       2.0      66.0       NaN       4.0   
3      93706.0      10.0       2.0       1.0      18.0       NaN       5.0   
4      93707.0      10.0       2.0       1.0      13.0       NaN       5.0   
...        ...       ...       ...       ...       ...       ...       ...   
9249  102952.0      10.0       2.0       2.0      70.0       NaN       5.0   
9250  102953.0      10.0       2.0       1.0      42.0       NaN       1.0   
9251  102954.0      10.0       2.0       2.0      41.0       NaN       4.0   
9252  102955.0      10.0       2.0       2.0      14.0       NaN       4.0   
9253  102956.0      10.0       2.0       1.0      38.0       NaN       3.0   

      RIDRETH3  RIDEXMON  RIDEXAGM  ...  DMDH

### Convert the XPTs into CSV

Consult: https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/DEMO_J.htm#RIDRETH1

For the reference information about the data

##### demo xpt

In [3]:
# pre process CSV
import numpy as np # For np.nan if needed for missing values

df_demo_raw = read_xpt("DEMO_J.XPT")

if not df_demo_raw.empty:
    # Select and Rename Columns

    required_nhanes_cols = {
        'SEQN': 'SEQN',
        'RIDAGEYR': 'Age',
        'RIAGENDR': 'Sex_Code',
        'RIDRETH3': 'RaceEthnicity_Code'
    }

    # Check if all required columns exist
    missing_cols = [col for col in required_nhanes_cols.keys() if col not in df_demo_raw.columns]
    if missing_cols:
        print(f"Error: Missing expected columns in DEMO_J.XPT: {missing_cols}")
        # Handle error appropriately, e.g., skip this transformation
    else:
        df_users = df_demo_raw[list(required_nhanes_cols.keys())].copy()
        df_users.rename(columns=required_nhanes_cols, inplace=True)

        # Transform Data

        # Recode Sex (RIAGENDR: 1=Male, 2=Female)
        sex_map = {1: 'M', 2: 'F'}
        df_users['Sex'] = df_users['Sex_Code'].map(sex_map)

        # Recode race
        race_map = {
            1: 'Mexican American',
            2: 'Other Hispanic',
            3: 'Non-Hispanic White',
            4: 'Non-Hispanic Black',
            6: 'Non-Hispanic Asian', # skips 5
            7: 'Other Race - Including Multi-Racial' # Or simply 'Other'
        }
        df_users['RaceEthnicity'] = df_users['RaceEthnicity_Code'].map(race_map)

        # Select final columns for users.csv and handle missing values
        # (UserID is auto-generated by the database)
        final_user_columns = ['SEQN', 'Age', 'Sex', 'RaceEthnicity']
        df_users_final = df_users[final_user_columns].copy()

        # Drop rows if SEQN is missing (it shouldn't be) or if essential fields like Sex are missing after mapping
        df_users_final.dropna(subset=['SEQN', 'Sex', 'RaceEthnicity', 'Age'], inplace=True)

        # Save to CSV
        clean_data_dir = Path("../data/clean")
        clean_data_dir.mkdir(parents=True, exist_ok=True) # Ensure the directory exists
        users_csv_path = clean_data_dir / "users.csv"

        df_users_final.to_csv(users_csv_path, index=False)
        print(f"\nSuccessfully created and saved users.csv to {users_csv_path}")
        print(df_users_final.head())
        df_users_final.info()


Successfully created and saved users.csv to ../data/clean/users.csv
      SEQN   Age Sex                        RaceEthnicity
0  93703.0   2.0   F                   Non-Hispanic Asian
1  93704.0   2.0   M                   Non-Hispanic White
2  93705.0  66.0   F                   Non-Hispanic Black
3  93706.0  18.0   M                   Non-Hispanic Asian
4  93707.0  13.0   M  Other Race - Including Multi-Racial
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9254 entries, 0 to 9253
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SEQN           9254 non-null   float64
 1   Age            9254 non-null   float64
 2   Sex            9254 non-null   object 
 3   RaceEthnicity  9254 non-null   object 
dtypes: float64(2), object(2)
memory usage: 289.3+ KB


In [4]:
# Set exam date
if 'RIDEXMON' in df_demo_raw.columns:
    df_exam_month_info = df_demo_raw[['SEQN', 'RIDEXMON']].copy()
APPROX_EXAM_DATE = "2018-01-01"

def assign_exam_date(row):
        if pd.isna(row['RIDEXMON']):
            return APPROX_EXAM_DATE

        year = "2018" # Simplified: Defaulting to the latter year of the cycle

        if row['RIDEXMON'] == 1.0: # Nov-Apr window
            # Let's represent this as Jan of the later year of the cycle.
            return f"2018-01-15"
        elif row['RIDEXMON'] == 2.0: # May-Oct window
            # Let's represent this as July of the later year of the cycle.
            return f"2018-07-15"
        return APPROX_EXAM_DATE

def populate_date(df, date_name):
     if not df_demo_raw.empty and 'RIDEXMON' in df_demo_raw.columns and 'SEQN' in df_demo_raw.columns:
        df_exam_month_info = df_demo_raw[['SEQN', 'RIDEXMON']].copy()
        # Merge RIDEXMON information into df
        df = pd.merge(df, df_exam_month_info, on='SEQN', how='left')
        
        # Apply the function to create the date_name column
        # The `axis=1` tells pandas to apply the function row-wise
        df[date_name] = df.apply(assign_exam_date, axis=1)
        
        # Convert the date_name to the desired string format
        df[date_name] = pd.to_datetime(df[date_name]).dt.strftime('%Y-%m-%d')
        print("Applied assign_exam_date() using RIDEXMON.")
        
        # Clean up RIDEXMON if it was merged and no longer needed directly in sessions.csv
        if 'RIDEXMON' in df.columns:
            df.drop(columns=['RIDEXMON'], inplace=True)
        return df[date_name]
     else:
        # Fallback if RIDEXMON or df_demo_raw is not available
        df[date_name] = APPROX_EXAM_DATE
        df[date_name] = pd.to_datetime(df[date_name]).dt.strftime('%Y-%m-%d')
        print("RIDEXMON not available or df_demo_raw missing; using general APPROX_EXAM_DATE for date_name.")
        return df[date_name]


##### BMX_J

Consult: https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/BMX_J.htm#RIDRETH1


In [5]:
# --- Processing BMX_J.XPT for anthropometry.csv ---
# Relevant BMX_J variables: SEQN, BMXWT, BMXHT, BMXBMI
df_bmx_raw = read_xpt("BMX_J.XPT", limit_cols=['SEQN', 'BMXWT', 'BMXHT', 'BMXBMI'])

if not df_bmx_raw.empty and 'SEQN' in df_bmx_raw.columns:
    df_anthropometry = df_bmx_raw.copy()
    
    # Rename columns to match schema
    rename_map_anthro = {
        'BMXWT': 'WeightKG',
        'BMXHT': 'HeightCM',
        'BMXBMI': 'BMI'
    }
    # Only rename columns that exist
    existing_rename_cols = {k: v for k, v in rename_map_anthro.items() if k in df_anthropometry.columns}
    df_anthropometry.rename(columns=existing_rename_cols, inplace=True)
    
    df_anthropometry['ExamDate'] = populate_date(df_anthropometry, 'ExamDate')

    # Select and order final columns
    # UserID in the database will be based on SEQN. We load SEQN here.
    final_anthro_columns = ['SEQN', 'ExamDate', 'HeightCM', 'WeightKG', 'BMI']
    
    # Filter for columns that actually exist in the DataFrame after renaming and adding
    existing_final_anthro_cols = [col for col in final_anthro_columns if col in df_anthropometry.columns]
    df_anthropometry_final = df_anthropometry[existing_final_anthro_cols]
    
    # Drop rows where SEQN is missing, or all measurement values are missing
    df_anthropometry_final.dropna(subset=['SEQN'], inplace=True)

    if not df_anthropometry_final.empty:
        anthropometry_csv_path = clean_data_dir / "anthropometry.csv"
        df_anthropometry_final.to_csv(anthropometry_csv_path, index=False)
        print(f"\nSuccessfully created and saved anthropometry.csv to {anthropometry_csv_path}")
        print(f"Anthropometry.csv has {len(df_anthropometry_final)} rows.")
    else:
        print("Could not generate anthropometry.csv due to missing columns or data.")
else:
    print("Skipping anthropometry.csv creation as BMX_J.XPT could not be read or SEQN is missing.")



Applied assign_exam_date() using RIDEXMON.

Successfully created and saved anthropometry.csv to ../data/clean/anthropometry.csv
Anthropometry.csv has 8704 rows.


#### Sessions

Consult: https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/FASTQX_J.htm#SDDSRVYR

In [6]:
# --- Processing FASTQX_J.XPT for sessions.csv ---

df_fasting_raw = read_xpt("FASTQX_J.XPT", limit_cols=['SEQN', 'PHAFSTHR', 'PHAFSTMN'])
df_fasting_status = pd.DataFrame() # Initialize

if not df_fasting_raw.empty and 'SEQN' in df_fasting_raw.columns:
    df_fasting = df_fasting_raw.copy()

    # Convert fasting hours and minutes to numeric, coercing errors to NaN
    # NHANES uses special codes for "Refused" or "Don't know" which will become NaN here.
    # PHAFSTHR: Hours (0-99)
    # PHAFSTMN: Minutes (0-59)
    for col in ['PHAFSTHR', 'PHAFSTMN']:
        if col in df_fasting.columns:
            df_fasting[col] = pd.to_numeric(df_fasting[col], errors='coerce')
        else:
            print(f"Warning: Expected fasting column {col} not found in FASTQX_J.XPT. Setting to 0.")
            df_fasting[col] = 0 # Add column as 0 if missing to prevent later errors

    # Fill NaN with 0 for calculation if you want to consider missing components as 0 duration
    # Or, if any part is NaN, the total duration could be NaN. Let's default to 0.
    df_fasting.fillna({'PHAFSTHR': 0, 'PHAFSTMN': 0}, inplace=True)

    # Calculate total fasting time in hours
    df_fasting['TotalFastingHours'] = df_fasting['PHAFSTHR'] + (df_fasting['PHAFSTMN'] / 60.0)

    # Define "fasting" criteria (e.g., >= 8 hours is common for glucose tests)
    # Your schema expects a BOOLEAN (0 or 1 for MySQL)
    fasting_threshold_hours = 8
    df_fasting['IsFasted'] = np.where(df_fasting['TotalFastingHours'] >= fasting_threshold_hours, 1, 0)
    
    # Select SEQN and the IsFasted column
    df_fasting_status = df_fasting[['SEQN', 'IsFasted']].copy()
    df_fasting_status.drop_duplicates(subset=['SEQN'], keep='first', inplace=True) # Should be one entry per SEQN

else:
    print("FASTQX_J.XPT could not be read or SEQN is missing. FastingStatus will use default.")
    # df_fasting_status will remain empty if the file isn't read

# Creating & Updating sessions.csv
if not df_users_final.empty and 'SEQN' in df_users_final.columns:
    df_sessions = df_users_final[['SEQN']].copy()
    # df_sessions.rename(columns={'SEQN': 'UserID_SEQN'}, inplace=True) # Using SEQN as the key for UserID
    
    df_sessions['SessionDate'] = populate_date(df_sessions, 'SessionDate')

    # Merge with fasting status information
    if not df_fasting_status.empty:
        df_sessions = pd.merge(df_sessions, df_fasting_status, on='SEQN', how='left')
        # If a participant from df_users_final is not in df_fasting_status, 'IsFasted' will be NaN.
        # Fill NaN 'IsFasted' with a default (e.g., 0 for not fasted/unknown)
        df_sessions['IsFasted'].fillna(0, inplace=True)
        df_sessions['FastingStatus'] = df_sessions['IsFasted'].astype(int)
        df_sessions.drop(columns=['IsFasted'], inplace=True) # Clean up intermediate column
    else:
        # Default if fasting data is unavailable
        df_sessions['FastingStatus'] = 0 

    # The schema `load.sh` expects columns named (UserID, SessionDate, FastingStatus)
    # where UserID in the CSV refers to the User.SEQN (assuming UserID in DB = SEQN or handled by load script)
    # The `longevity-biomarker-tracker/etl/load.sh` script loads `sessions.csv` columns (UserID, SessionDate, FastingStatus)
    # So, the column in sessions.csv that holds the SEQN value should be named 'UserID'
    df_sessions.rename(columns={'SEQN': 'UserID'}, inplace=True) # Rename SEQN to UserID for the CSV
    
    final_session_columns = ['UserID', 'SessionDate', 'FastingStatus']
    # Ensure all necessary columns are present
    existing_final_session_cols = [col for col in final_session_columns if col in df_sessions.columns]
    
    if len(existing_final_session_cols) == len(final_session_columns):
        df_sessions_final = df_sessions[existing_final_session_cols]

        if not df_sessions_final.empty:
            sessions_csv_path = clean_data_dir / "sessions.csv"
            df_sessions_final.to_csv(sessions_csv_path, index=False)
            print(f"\nSuccessfully created and saved sessions.csv to {sessions_csv_path} (with fasting data if available).")
            print(f"Sessions.csv has {len(df_sessions_final)} rows.")
            print(df_sessions_final.head())
        else:
            print("Could not generate sessions.csv (empty after processing).")
            df_sessions_final = pd.DataFrame() # Ensure it's defined
    else:
        missing_cols_for_sessions = set(final_session_columns) - set(existing_final_session_cols)
        print(f"Could not generate sessions.csv due to missing columns: {missing_cols_for_sessions}")
        df_sessions_final = pd.DataFrame() # Ensure it's defined
        
else:
    df_sessions_final = pd.DataFrame()
    print("Skipping sessions.csv creation as users.csv (df_users_final) is not available or SEQN is missing.")

Applied assign_exam_date() using RIDEXMON.

Successfully created and saved sessions.csv to ../data/clean/sessions.csv (with fasting data if available).
Sessions.csv has 9254 rows.
    UserID SessionDate  FastingStatus
0  93703.0  2018-07-15              0
1  93704.0  2018-01-15              0
2  93705.0  2018-07-15              0
3  93706.0  2018-07-15              1
4  93707.0  2018-07-15              0


/var/folders/2j/8wxk37tn5dlcg09q_bz7zmtm0000gn/T/ipykernel_88573/3370105991.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sessions['IsFasted'].fillna(0, inplace=True)


#### Processing for measurements

In [7]:
# --- Processing several XPTs for measurements.csv ---

biomarker_map_nhanes_to_id = {
    'LBXSAL': 1,  # Albumin
    'LBXSAPSI': 2, # Alkaline Phosphatase
    'LBXSCR': 3,  # Creatinine
    'LBXGLU': 4,  # Fasting Glucose - This comes from GLU_J.XPT, others from BIOPRO_J/CBC_J
    'LBXHSCRP': 5, # High-Sensitivity CRP - This comes from HSCRP_J.XPT
    'LBXWBCSI': 6, # White Blood Cell Count
    'LBXLYPCT': 7, # Lymphocyte Percentage
    'LBXMCVSI': 8, # Mean Corpuscular Volume
    'LBXRDW': 9    # Red Cell Distribution Width
}
# Create a reverse map for convenience if needed: id_to_nhanes_code
id_to_nhanes_code = {v: k for k, v in biomarker_map_nhanes_to_id.items()}

lab_files_config = {
    "BIOPRO_J.XPT": ['SEQN', 'LBXSAL', 'LBXSAPSI', 'LBXSCR'], # Albumin, Alk Phos, Creatinine
    "GLU_J.XPT": ['SEQN', 'LBXGLU'],                          # Fasting Glucose. Also check LBDGLUSI for SI units if preferred.
                                                              # GLU_J also has WTSAF2YR (fasting subsample weight)
                                                              # and PHAFSTHR/MN (fasting time) are in FASTQX_J
    "HSCRP_J.XPT": ['SEQN', 'LBXHSCRP'],                      # HS CRP (variable name is indeed LBXHSCRP in docs)
    "CBC_J.XPT": ['SEQN', 'LBXWBCSI', 'LBXLYPCT', 'LBXMCVSI', 'LBXRDW'] # WBC, Lymph %, MCV, RDW
}

all_measurements_list = []

for file_name, nhanes_vars in lab_files_config.items():
    df_lab_raw = read_xpt(file_name, limit_cols=nhanes_vars)
    
    if df_lab_raw.empty or 'SEQN' not in df_lab_raw.columns:
        print(f"Skipping {file_name} due to read error or missing SEQN.")
        continue
        
    # Melt the DataFrame to long format: SEQN, NHANESVarCode, Value
    # Keep only SEQN and the actual biomarker columns present in nhanes_vars AND df_lab_raw
    biomarker_cols_in_file = [var for var in nhanes_vars if var != 'SEQN' and var in df_lab_raw.columns]
    if not biomarker_cols_in_file:
        print(f"No specified biomarker columns found in {file_name}.")
        continue

    df_melted = df_lab_raw.melt(id_vars=['SEQN'], 
                                value_vars=biomarker_cols_in_file,
                                var_name='NHANESVarCode', 
                                value_name='Value')
    
    # Drop rows where Value is NaN (missing measurement)
    df_melted.dropna(subset=['Value'], inplace=True)
    
    if df_melted.empty:
        print(f"No valid measurements after melting and dropping NaNs for {file_name}.")
        continue

    all_measurements_list.append(df_melted)

if all_measurements_list:
    df_all_measurements = pd.concat(all_measurements_list, ignore_index=True)
    
    # Map NHANESVarCode to BiomarkerID
    df_all_measurements['BiomarkerID'] = df_all_measurements['NHANESVarCode'].map(biomarker_map_nhanes_to_id)
    
    # Drop rows where BiomarkerID could not be mapped (i.e., NHANESVarCode not in our map)
    df_all_measurements.dropna(subset=['BiomarkerID'], inplace=True)
    df_all_measurements['BiomarkerID'] = df_all_measurements['BiomarkerID'].astype(int)
    
    # Add TakenAt date
    df_all_measurements['TakenAt'] =  populate_date(df_all_measurements, 'TakenAt')
        
    df_measurements_final = df_all_measurements[['SEQN', 'BiomarkerID', 'Value', 'TakenAt']].copy()
    df_measurements_final.dropna(inplace=True) # Ensure all rows are complete

    if not df_measurements_final.empty:
        measurements_csv_path = clean_data_dir / "measurements.csv"
        df_measurements_final.to_csv(measurements_csv_path, index=False)
        print(f"\nSuccessfully created and saved measurements.csv to {measurements_csv_path}")
        print(f"Measurements.csv has {len(df_measurements_final)} rows.")
    else:
        print("Could not generate measurements.csv due to missing data or mapping issues.")
else:
    print("No lab data processed for measurements.csv.")

print("\n--- ETL Transformation Script Complete ---")
print(f"CSVs generated in: {clean_data_dir}")
print("Please review the generated CSVs.")
print("Note: The measurements.csv currently contains SEQN. The load.sh script will need to be adapted")
print("to look up the correct auto-generated SessionID from the MeasurementSession table based on this SEQN (as UserID) and SessionDate.")

Applied assign_exam_date() using RIDEXMON.

Successfully created and saved measurements.csv to ../data/clean/measurements.csv
Measurements.csv has 57959 rows.

--- ETL Transformation Script Complete ---
CSVs generated in: ../data/clean
Please review the generated CSVs.
Note: The measurements.csv currently contains SEQN. The load.sh script will need to be adapted
to look up the correct auto-generated SessionID from the MeasurementSession table based on this SEQN (as UserID) and SessionDate.


In [8]:
# === FINAL PATCH CELL: Fix CSV formats for database loading ===
# (Added to make CSVs compatible with load.sh without changing existing logic)

print("=== Applying final patches for database compatibility ===")

# A. Fix users.csv: Replace Age with BirthDate
if 'df_users_final' in globals() and not df_users_final.empty:
    CYCLE_YEAR = 2018  # NHANES J cycle is 2017-2018

    def derive_birthdate(age):
        return f"{CYCLE_YEAR - int(age)}-07-01"  # Mid-year birthdate

    df_users_final["BirthDate"] = df_users_final["Age"].apply(derive_birthdate)
    df_users_final.drop(columns=["Age"], inplace=True)

    # Re-export users.csv
    users_csv_path = clean_data_dir / "users.csv"
    df_users_final.to_csv(users_csv_path, index=False)
    print(f"✓ Updated users.csv with BirthDate column")

# B. Fix sessions.csv: Change UserID back to SEQN
if 'df_sessions_final' in globals() and not df_sessions_final.empty:
    df_sessions_final.rename(columns={"UserID": "SEQN"}, inplace=True)

    # Re-export sessions.csv
    sessions_csv_path = clean_data_dir / "sessions.csv"
    df_sessions_final.to_csv(sessions_csv_path, index=False)
    print(f"✓ Updated sessions.csv with SEQN column")

# C. Fix measurements.csv: Add SessionDate for join
if 'df_measurements_final' in globals() and not df_measurements_final.empty:
    # Add SessionDate (same as TakenAt for simplicity)
    df_measurements_final["SessionDate"] = df_measurements_final["TakenAt"]

    # Reorder columns for load.sh
    final_cols = ["SEQN", "SessionDate", "BiomarkerID", "Value", "TakenAt"]
    df_measurements_final = df_measurements_final[final_cols]

    # Re-export measurements.csv
    measurements_csv_path = clean_data_dir / "measurements.csv"
    df_measurements_final.to_csv(measurements_csv_path, index=False)
    print(f"✓ Updated measurements.csv with SessionDate column")

print("\n=== CSV Compatibility Patches Complete ===")
print("All CSVs should now load successfully with load.sh")
print(f"Files updated in: {clean_data_dir}")

# === D: align users.csv with load.sh column order ===
print("⏩  Final users.csv patch – order & SEQN dtype")

if "df_users_final" in globals() and not df_users_final.empty:
    # Ensure BirthDate already exists (previous patch)
    if "BirthDate" not in df_users_final.columns:
        raise ValueError("BirthDate column missing – run earlier cells first")

    # 1) Cast SEQN to int so we don't get 93703.0 in the CSV
    df_users_final["SEQN"] = df_users_final["SEQN"].astype(int)

    # 2) Re-order columns to match LOAD DATA list
    df_users_final = df_users_final[["SEQN", "BirthDate", "Sex", "RaceEthnicity"]]

    # 3) Overwrite the CSV
    users_csv_path = clean_data_dir / "users.csv"
    df_users_final.to_csv(users_csv_path, index=False)
    print("✓ users.csv fixed →", users_csv_path)

# Display final column structure for verification
for csv_file in ["users.csv", "sessions.csv", "measurements.csv", "anthropometry.csv"]:
    csv_path = clean_data_dir / csv_file
    if csv_path.exists():
        df_check = pd.read_csv(csv_path)
        print(f"\n{csv_file} columns: {list(df_check.columns)}")
        print(f"  Rows: {len(df_check)}")
    else:
        print(f"\n{csv_file}: Not found")

=== Applying final patches for database compatibility ===
✓ Updated users.csv with BirthDate column
✓ Updated sessions.csv with SEQN column
✓ Updated measurements.csv with SessionDate column

=== CSV Compatibility Patches Complete ===
All CSVs should now load successfully with load.sh
Files updated in: ../data/clean
⏩  Final users.csv patch – order & SEQN dtype
✓ users.csv fixed → ../data/clean/users.csv

users.csv columns: ['SEQN', 'BirthDate', 'Sex', 'RaceEthnicity']
  Rows: 9254

sessions.csv columns: ['SEQN', 'SessionDate', 'FastingStatus']
  Rows: 9254

measurements.csv columns: ['SEQN', 'SessionDate', 'BiomarkerID', 'Value', 'TakenAt']
  Rows: 57959

anthropometry.csv columns: ['SEQN', 'ExamDate', 'HeightCM', 'WeightKG', 'BMI']
  Rows: 8704
